In [1]:
from PIL import Image
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

In [2]:
metadata_df = pd.DataFrame(columns=["Path","Class","White_noise","Pepper_noise"])
metadata_df

,Path,Class,White_noise,Pepper_noise


In [3]:
def add_impulse_noise(image, prob_salt=0.1, prob_pepper=0.02):
    if isinstance(image, Image.Image):
        img_array = np.array(image)
    else:
        img_array = image.copy()

    noisy_image = img_array.copy()
    total_pixels = img_array.size
    num_salt = int(total_pixels * prob_salt)
    num_pepper = int(total_pixels * prob_pepper)
    coords = [np.random.randint(0, i - 1, num_salt) for i in img_array.shape]
    noisy_image[coords[0], coords[1]] = 255
    coords = [np.random.randint(0, i - 1, num_pepper) for i in img_array.shape]
    noisy_image[coords[0], coords[1]] = 0
    if len(img_array.shape) == 3:
        noisy_image[coords[0], coords[1], :] = 0
    return Image.fromarray(noisy_image)

In [4]:
PATH = '/Users/vedanshkumar/Documents/Fun_ml/Projects/FuzzyLogicFiltering/dataset/AID'
CATEGORIES = os.listdir(PATH)

In [5]:
# Ensure directories exist
clean_dir = "/Users/vedanshkumar/Documents/Fun_ml/Projects/FuzzyLogicFiltering/dataset/Dataset/Clean"
noisy_dir = "/Users/vedanshkumar/Documents/Fun_ml/Projects/FuzzyLogicFiltering/dataset/Dataset/Noisy"

os.makedirs(clean_dir, exist_ok=True)
os.makedirs(noisy_dir, exist_ok=True)

for category in CATEGORIES:
    imgs = os.listdir(f'{PATH}/{category}')
    
    for img in imgs:
        img_path = f'{PATH}/{category}/{img}'
        clean_img = Image.open(img_path)

        # Add random salt and pepper noise
        prob_salt = random.uniform(0.01, 0.06)
        prob_pepper = random.uniform(0.01, 0.06)
        
        noisy_img = add_impulse_noise(clean_img, prob_salt, prob_pepper)

        # Save images
        clean_img.save(f"{clean_dir}/{img}")
        noisy_img.save(f"{noisy_dir}/noisy_{img}")

        # Append metadata using pd.concat()
        new_data = pd.DataFrame([
            {"Path": f"Clean/{img}", "Class": "Clean", "White_noise": 0, "Pepper_noise": 0},
            {"Path": f"Noisy/noisy_{img}", "Class": "Noisy", "White_noise": prob_salt, "Pepper_noise": prob_pepper}
        ])
        
        metadata_df = pd.concat([metadata_df, new_data], ignore_index=True)

/var/folders/kt/rpgv1_gs2sz58ldgmhqpxdfc0000gn/T/ipykernel_24389/4073754819.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata_df = pd.concat([metadata_df, new_data], ignore_index=True)


In [6]:
metadata_df.columns

Index(['Path', 'Class', 'White_noise', 'Pepper_noise'], dtype='object')

In [7]:
metadata_df

,Path,Class,White_noise,Pepper_noise
0,Clean/pond_127.jpg,Clean,0.000000,0.000000
1,Noisy/noisy_pond_127.jpg,Noisy,0.057622,0.048258
2,Clean/pond_133.jpg,Clean,0.000000,0.000000
3,Noisy/noisy_pond_133.jpg,Noisy,0.028439,0.025684
4,Clean/pond_319.jpg,Clean,0.000000,0.000000
...,...,...,...,...
19995,Noisy/noisy_storagetanks_38.jpg,Noisy,0.015900,0.058787
19996,Clean/storagetanks_236.jpg,Clean,0.000000,0.000000
19997,Noisy/noisy_storagetanks_236.jpg,Noisy,0.022757,0.058457
19998,Clean/storagetanks_222.jpg,Clean,0.000000,0.000000


In [8]:
metadata_df.to_csv(r"/Users/vedanshkumar/Documents/Fun_ml/Projects/FuzzyLogicFiltering/dataset/Dataset/metadata.csv")